<h4>Добавьте туда файл conftest.py из предыдущего модуля. <br>Убедитесь дополнительно, что там есть параметр для задания языка интерфейса, по умолчанию равный "en".</h4>

In [1]:
%%writefile conftest.py

import pytest
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def pytest_addoption(parser):
    parser.addoption('--browser_name', action='store', default="chrome",
                     help="Choose browser: chrome or firefox")
    parser.addoption('--language', action='store', default="en",
                     help="Choose language")
    
@pytest.fixture(scope="function")
def language(request):
    language = request.config.getoption("language")
    return language 

@pytest.fixture(scope="function")
def browser(request,language):
    
    browser_name = request.config.getoption("browser_name")
    browser = None
    
    if browser_name == "chrome":
        
        print("\nstart chrome browser for test..")
        options = Options()
        options.add_experimental_option('prefs', {'intl.accept_languages': language})
        browser = webdriver.Chrome(options=options)
        
    elif browser_name == "firefox":
        
        print("\nstart firefox browser for test..")
        fp = webdriver.FirefoxProfile()
        fp.set_preference("intl.accept_languages", language)
        browser = webdriver.Firefox(firefox_profile=fp)

    else:
        
        raise pytest.UsageError("--browser_name should be chrome or firefox")
        
    yield browser
    print("\nquit browser..")
    browser.quit()

    

Overwriting conftest.py


In [2]:
%%writefile pages/locators.py

from selenium.webdriver.common.by import By

class MainPageLocators():
    LOGIN_LINK = (By.CSS_SELECTOR, "#login_link")
    
class LoginPageLocators():
    LOGIN_FORM = (By.CSS_SELECTOR, "#login_form")
    REGISTER_FORM = (By.CSS_SELECTOR, "#register_form")
    

Overwriting pages/locators.py


In [3]:
%%writefile pages/base_page.py

from selenium.common.exceptions import NoSuchElementException

class BasePage():
    
    def __init__(self, browser, url, timeout=10):
        
        self.browser = browser
        self.url = url
        self.browser.implicitly_wait(timeout)
        
        
    def open(self):
        
        self.browser.get(self.url)
        
        
    def is_element_present(self, how, what):
        
        try:
            self.browser.find_element(how, what)
        except NoSuchElementException:
            return False
        return True    

Overwriting pages/base_page.py


In [10]:
%%writefile pages/login_page.py

from .base_page import BasePage
from .locators import LoginPageLocators

class LoginPage(BasePage):
    
    def should_be_login_page(self):
        self.should_be_login_url()
        self.should_be_login_form()
        self.should_be_register_form()

    def should_be_login_url(self):
        
        assert "login" in self.browser.current_url, 'Incorrected URL (not inclusive substring "login").'

    def should_be_login_form(self):
        
        assert self.browser.find_element(*LoginPageLocators.LOGIN_FORM) ,'Up-s-s.Login form not found...'

    def should_be_register_form(self):
        
        assert self.browser.find_element(*LoginPageLocators.REGISTER_FORM),'Register form not found...'

Overwriting pages/login_page.py


In [5]:
%%writefile pages/main_page.py

from selenium.webdriver.common.by import By
from .base_page import BasePage
from .locators import MainPageLocators

class MainPage(BasePage): 
    
    def go_to_login_page(self):
        
        login_link = self.browser.find_element(*MainPageLocators.LOGIN_LINK)
        login_link.click()
            
    def should_be_login_link(self):
        assert self.is_element_present(*MainPageLocators.LOGIN_LINK), "Login link is not presented"
        

Overwriting pages/main_page.py


In [6]:
%%writefile test_main_page.py

from pages.main_page import MainPage

link = "http://selenium1py.pythonanywhere.com/"

class Test_main_page():
    
    def test_guest_should_see_login_link(self, browser):

        page = MainPage(browser, link)
        page.open()
        page.should_be_login_link()  


    def test_guest_can_go_to_login_page(self,browser):

        page = MainPage(browser, link)   
        page.open()
        page.go_to_login_page()

  

Overwriting test_main_page.py


In [11]:
!pytest -sv --tb=line --language=en test_main_page.py

============================= test session starts ==============================
platform darwin -- Python 3.10.0, pytest-6.2.5, py-1.11.0, pluggy-0.13.1 -- /Users/silinanata/environments/selenium_env/bin/python3
cachedir: .pytest_cache
rootdir: /Users/silinanata/selenium_project/first-selenium-project
plugins: rerunfailures-10.2
collected 2 items                                                              

test_main_page.py::Test_main_page::test_guest_should_see_login_link 
start chrome browser for test..
PASSED
quit browser..

test_main_page.py::Test_main_page::test_guest_can_go_to_login_page 
start chrome browser for test..
PASSED
quit browser..


============================== 2 passed in 12.32s ==============================
